In [1]:
# Import modules
import numpy as np
import pandas as pd
import seaborn as sns
import dc_stat_think as dcst

In [2]:
item = pd.read_csv('grocery/item_to_id.csv')
display(item.head())
purchase = pd.read_csv('grocery/purchase_history.csv')
display(purchase.head())

,Item_name,Item_id
0,coffee,43
1,tea,23
2,juice,38
3,soda,9
4,sandwich loaves,39


,user_id,id
0,222087,"27,26"
1,1343649,"6,47,17"
2,404134,"18,12,23,22,27,43,38,20,35,1"
3,1110200,"9,23,2,20,26,47,37"
4,224107,"31,18,5,13,1,21,48,16,26,2,44,32,20,37,42,35,4..."


In [5]:
# how many items do we have?
print('Company has', len(item['Item_name'].values), 'items.')
print()
print(item['Item_name'].values)

Company has 48 items.

['coffee' 'tea' 'juice' 'soda' 'sandwich loaves' 'dinner rolls'
 'tortillas' 'bagels' 'canned vegetables' 'spaghetti sauce' 'ketchup'
 'cheeses' 'eggs' 'milk' 'yogurt' 'butter' 'cereals' 'flour' 'sugar'
 'pasta' 'waffles' 'frozen vegetables' 'ice cream' 'poultry' 'beef' 'pork'
 'bananas' 'berries' 'cherries' 'grapefruit' 'apples' 'broccoli' 'carrots'
 'cauliflower' 'cucumbers' 'lettuce' 'laundry detergent' 'dishwashing\xa0'
 'paper towels' 'toilet paper' 'aluminum foil' 'sandwich bags' 'shampoo'
 'soap' 'hand soap' 'shaving cream' 'baby items' 'pet items']


In [10]:
# diswashing looks weird: 'diswashing\xa0'?
item['Item_name'] = item['Item_name'].str.replace('\xa0', '')
print(item['Item_name'].values)

['coffee' 'tea' 'juice' 'soda' 'sandwich loaves' 'dinner rolls'
 'tortillas' 'bagels' 'canned vegetables' 'spaghetti sauce' 'ketchup'
 'cheeses' 'eggs' 'milk' 'yogurt' 'butter' 'cereals' 'flour' 'sugar'
 'pasta' 'waffles' 'frozen vegetables' 'ice cream' 'poultry' 'beef' 'pork'
 'bananas' 'berries' 'cherries' 'grapefruit' 'apples' 'broccoli' 'carrots'
 'cauliflower' 'cucumbers' 'lettuce' 'laundry detergent' 'dishwashing'
 'paper towels' 'toilet paper' 'aluminum foil' 'sandwich bags' 'shampoo'
 'soap' 'hand soap' 'shaving cream' 'baby items' 'pet items']


# Merge two dataframes

In [17]:
purchase['id_split'] = purchase['id'].str.split(',')
purchase.head()

,user_id,id,id_split
0,222087,"27,26","[27, 26]"
1,1343649,"6,47,17","[6, 47, 17]"
2,404134,"18,12,23,22,27,43,38,20,35,1","[18, 12, 23, 22, 27, 43, 38, 20, 35, 1]"
3,1110200,"9,23,2,20,26,47,37","[9, 23, 2, 20, 26, 47, 37]"
4,224107,"31,18,5,13,1,21,48,16,26,2,44,32,20,37,42,35,4...","[31, 18, 5, 13, 1, 21, 48, 16, 26, 2, 44, 32, ..."


In [21]:
# Explode
purchase = purchase.explode('id_split')

# Reorganize the dataframe
purchase = purchase.drop('id', axis=1).reset_index(drop=True)
purchase.head()

,user_id,id_split
0,222087,27
1,222087,26
2,1343649,6
3,1343649,47
4,1343649,17


In [23]:
# Before merging the two dataframes on'id_split' and 'Item_id', need to make sure they have the same data type
display(item.info())
display(purchase.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Item_name  48 non-null     object
 1   Item_id    48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 896.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319995 entries, 0 to 319994
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   319995 non-null  int64 
 1   id_split  319995 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.9+ MB


None

In [24]:
# id_split is of type object, let's recast it to int
purchase['id_split'] = purchase['id_split'].astype('int')
display(purchase.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319995 entries, 0 to 319994
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   user_id   319995 non-null  int64
 1   id_split  319995 non-null  int64
dtypes: int64(2)
memory usage: 4.9 MB


None

In [27]:
full = purchase.merge(item, left_on='id_split', right_on='Item_id')
full = full.drop(labels=['id_split', 'Item_id'], axis=1)
display(full.head())

,user_id,Item_name
0,222087,dishwashing
1,404134,dishwashing
2,987613,dishwashing
3,60201,dishwashing
4,1166671,dishwashing


# Answer the questions

## Customer who bought the most items overall in her lifetime

In [45]:
# Top 5 customers who bought the most items
(full.groupby('user_id')
     .count().
     .rename(columns={'Item_name': 'number_of_purchase'})
     .reset_index()                                         # make user_id a field instead of being an index
     .sort_values(by='number_of_purchase', ascending=False) # sort customer with highest purchase items to the top
     .head()
)

,user_id,number_of_purchase
4445,269335,72
6111,367872,70
6623,397623,64
9918,599172,64
6291,377284,63


In [52]:
# Test
assert np.sum([full['user_id'] == 269335]) == 72

## For each item, the customer who bought that product the most
https://realpython.com/pandas-groupby/

In [110]:
# SELECT
#     Item_name,
#     user_id
#     COUNT(user_id)
# FROM full
# GROUP BY
#     Item_name,
#     user_id
# ORDER BY COUNT(user_id) DESCT

purchase_amount = \
(
    full.groupby(['Item_name', 'user_id'])['user_id']
    .count()
    .rename('dummy')
    .reset_index()
    .rename(columns={'dummy': 'number_of_purchase'})
    .sort_values(by=['Item_name', 'number_of_purchase'], ascending=(True, False))
)

rk = \
(
    purchase_amount
    .groupby(['Item_name'])['number_of_purchase'].rank(ascending=False, method='dense')
    .rename('rank')
)

purchase_amount = pd.concat([purchase_amount, rk], axis=1)
display(purchase_amount.head())

,Item_name,user_id,number_of_purchase,rank
306,aluminum foil,143741,3,1.0
3,aluminum foil,553,2,2.0
28,aluminum foil,13321,2,2.0
53,aluminum foil,21779,2,2.0
72,aluminum foil,33011,2,2.0


In [116]:
most_valuable_customer = purchase_amount[purchase_amount['rank'] == 1][['Item_name', 'user_id', 'number_of_purchase']]
most_valuable_customer

,Item_name,user_id,number_of_purchase
306,aluminum foil,143741,3
3694,apples,109578,4
5179,apples,384935,4
6071,apples,545108,4
10165,apples,1303742,4
...,...,...,...
289381,yogurt,1167089,3
289390,yogurt,1169085,3
289532,yogurt,1213479,3
289757,yogurt,1280108,3
